# Multiview

In [10]:
import os

data_root = '/data_new/chence/Ours/HeadNeRF/1024x1024'
export_root = '/data_new/chence/ExportedMVhq'
os.makedirs(export_root, exist_ok=True)

In [11]:
align_images_dir = os.path.join(export_root, 'align_images')
os.makedirs(align_images_dir, exist_ok=True)
align_parsing_dir = os.path.join(export_root, 'align_parsing')
os.makedirs(align_parsing_dir, exist_ok=True)
head_images_dir = os.path.join(export_root, 'head_images')
os.makedirs(head_images_dir, exist_ok=True)
head_parsing_dir = os.path.join(export_root, 'head_parsing')
os.makedirs(head_parsing_dir, exist_ok=True)

In [14]:
import json
import os.path as osp

def load_json(json_path):
    with open(json_path, 'r') as f:
        json_data = json.load(f)
    return json_data


export_start_idx = 0
export_meta_json = os.path.join(export_root, f'meta_{export_start_idx:07d}.json')
export_meta_dict = {}
export_instance_json = os.path.join(export_root, 'instance.json')
if osp.exists(export_instance_json):
    print("load instance dict.")
    with open(export_instance_json, 'r') as f:
        export_instance_dict = json.load(f)
else:
    export_instance_dict = {}

In [15]:
def chk_cldir(cldir_path):
    align_image_path = osp.join(cldir_path, 'align_image')
    if not (osp.exists(align_image_path) and osp.isdir(align_image_path)): return False
    align_parsing_path = osp.join(cldir_path, 'align_parsing')
    if not (osp.exists(align_parsing_path) and osp.isdir(align_parsing_path)): return False
    head_image_path = osp.join(cldir_path, 'head_image')
    if not (osp.exists(head_image_path) and osp.isdir(head_image_path)): return False
    head_parsing_path = osp.join(cldir_path, 'head_parsing')
    if not (osp.exists(head_parsing_path) and osp.isdir(head_parsing_path)): return False
    meta_path = osp.join(cldir_path, 'meta.json')
    if not (osp.exists(meta_path) and osp.isfile(meta_path)): return False

    align_images = sorted([i for i in os.listdir(align_image_path) if osp.isfile(osp.join(align_image_path, i))])
    if not (len(align_images) == 30): return False
    align_parsings = sorted([i for i in os.listdir(align_parsing_path) if osp.isfile(osp.join(align_parsing_path, i))])
    if not (len(align_parsings) == 30): return False
    head_images = sorted([i for i in os.listdir(head_image_path) if osp.isfile(osp.join(head_image_path, i))])
    if not (len(head_images) == 30): return False
    head_parsings = sorted([i for i in os.listdir(head_parsing_path) if osp.isfile(osp.join(head_parsing_path, i))])
    if not (len(head_parsings) == 30): return False

    if not (align_images == align_parsings): return False
    if not (align_images == head_images): return False
    if not (head_images == head_parsings): return False

    return True


def isProcessed(datadir):
    chgclres_path = osp.join(datadir, 'change_color_result')
    if not (osp.exists(chgclres_path) and osp.isdir(chgclres_path)): return False
    cldirs = sorted([i for i in os.listdir(chgclres_path) if osp.isdir(osp.join(chgclres_path, i))])
    if not (len(cldirs) == 6): return False # 5 + 1
    if not ('Original' in cldirs): return False
    for i in cldirs:
        cldir_path = osp.join(chgclres_path, i)
        if not (osp.exists(cldir_path) and osp.isdir(cldir_path)): return False
        if not (chk_cldir(cldir_path)): return False
    return True


import tqdm

datadirs = []
for i in tqdm.tqdm(os.listdir(data_root)):
    cur_datadir = osp.join(data_root, i, '00/recolornerf/panohead')
    if osp.exists(cur_datadir) and osp.isdir(cur_datadir):
        if isProcessed(cur_datadir): datadirs.append(cur_datadir)
datadirs = sorted(datadirs)

  0%|          | 0/856 [00:00<?, ?it/s]

100%|██████████| 856/856 [00:11<00:00, 74.12it/s] 


In [16]:
print(len(datadirs))

824


In [17]:
instance_idx = len(export_instance_dict)
relative_root = '/data_new/chence'
instance_image_idx = {}
for datadir in tqdm.tqdm(datadirs):
    chgclres_path = osp.join(datadir, 'change_color_result')
    cldirs = sorted([i for i in os.listdir(chgclres_path) if osp.isdir(osp.join(chgclres_path, i))])
    for i in cldirs:
        cldir_path = osp.join(chgclres_path, i)
        instance_key = osp.relpath(cldir_path, relative_root)
        if instance_key not in export_instance_dict.keys():
            export_instance_dict[instance_key] = str(instance_idx).zfill(5)
            instance_idx += 1
        export_folder = export_instance_dict[instance_key]
        meta_path = osp.join(cldir_path, 'meta.json')
        meta_data = load_json(meta_path)
        meta_keys = sorted(meta_data.keys())
        for k in meta_keys:
            align_image_path = osp.join(cldir_path, 'align_image', k)
            assert osp.exists(align_image_path) and osp.isfile(align_image_path)
            align_parsing_path = osp.join(cldir_path, 'align_parsing', k)
            assert osp.exists(align_parsing_path) and osp.isfile(align_parsing_path)
            head_image_path = osp.join(cldir_path, 'head_image', k)
            assert osp.exists(head_image_path) and osp.isfile(head_image_path)
            head_parsing_path = osp.join(cldir_path, 'head_parsing', k)
            assert osp.exists(head_parsing_path) and osp.isfile(head_parsing_path)

            if export_folder not in instance_image_idx.keys():
                instance_image_idx[export_folder] = 0
            export_name = 'img'+str(instance_image_idx[export_folder]).zfill(8)+'.png'
            instance_image_idx[export_folder] += 1
            export_key = f'{export_folder}/{export_name}'
            export_meta_dict[export_key] = meta_data[k]
            export_meta_dict[export_key].update({
                'align_image_path': osp.relpath(align_image_path, relative_root),
                'align_parsing_path': osp.relpath(align_parsing_path, relative_root),
                'head_image_path': osp.relpath(head_image_path, relative_root),
                'head_parsing_path': osp.relpath(head_parsing_path, relative_root),
            })
            export_start_idx += 1

100%|██████████| 824/824 [00:50<00:00, 16.32it/s]


In [18]:
print(len(instance_image_idx))
print(len(export_meta_dict))
assert len(instance_image_idx) * 30 == len(export_meta_dict)

4944
148320


In [19]:
list(export_instance_dict.keys())[-5]

'Ours/HeadNeRF/1024x1024/1197/00/recolornerf/panohead/change_color_result/Dark Maroon'

In [20]:
with open(export_meta_json, 'w') as f:
    json.dump(export_meta_dict, f, indent=4)

with open(export_instance_json, 'w') as f:
    json.dump(export_instance_dict, f, indent=4)

print(len(export_meta_dict))

148320


In [23]:
import cv2

def work(export_key, data_item, data_root, export_align_images_dir, export_align_parsing_dir, export_head_images_dir, export_head_parsing_dir):
    try:
        export_folder = export_key.split('/')[0]
        export_align_image_folder = os.path.join(export_align_images_dir, export_folder)
        export_align_parsing_folder = os.path.join(export_align_parsing_dir, export_folder)
        export_head_image_folder = os.path.join(export_head_images_dir, export_folder)
        export_head_parsing_folder = os.path.join(export_head_parsing_dir, export_folder)
        os.makedirs(export_align_image_folder, exist_ok=True)
        os.makedirs(export_align_parsing_folder, exist_ok=True)
        os.makedirs(export_head_image_folder, exist_ok=True)
        os.makedirs(export_head_parsing_folder, exist_ok=True)

        export_align_image_path = os.path.join(export_align_images_dir, export_key)
        export_align_parsing_path = os.path.join(export_align_parsing_dir, export_key)
        export_head_image_path = os.path.join(export_head_images_dir, export_key)
        export_head_parsing_path = os.path.join(export_head_parsing_dir, export_key)

        cur_align_image_path = os.path.join(data_root, data_item['align_image_path'])
        assert os.path.exists(cur_align_image_path)
        if cur_align_image_path.endswith('.png') or cur_align_image_path.endswith('.PNG'):
            os.system(f'cp "{cur_align_image_path}" "{export_align_image_path}"')
        else:
            cur_align_image_data = cv2.imread(cur_align_image_path)
            cv2.imwrite(export_align_image_path, cur_align_image_data)

        cur_align_parsing_path = os.path.join(data_root, data_item['align_parsing_path'])
        assert os.path.exists(cur_align_parsing_path)
        assert cur_align_parsing_path.endswith('.png') or cur_align_parsing_path.endswith('.PNG')
        os.system(f'cp "{cur_align_parsing_path}" "{export_align_parsing_path}"')

        cur_head_image_path = os.path.join(data_root, data_item['head_image_path'])
        assert os.path.exists(cur_head_image_path)
        if cur_head_image_path.endswith('.png') or cur_head_image_path.endswith('.PNG'):
            os.system(f'cp "{cur_head_image_path}" "{export_head_image_path}"')
        else:
            cur_head_image_data = cv2.imread(cur_head_image_path)
            cv2.imwrite(export_head_image_path, cur_head_image_data)

        cur_head_parsing_path = os.path.join(data_root, data_item['head_parsing_path'])
        assert os.path.exists(cur_head_parsing_path)
        assert cur_head_parsing_path.endswith('.png') or cur_head_parsing_path.endswith('.PNG')
        os.system(f'cp "{cur_head_parsing_path}" "{export_head_parsing_path}"')
    except Exception as e:
        print(e)

In [25]:
import multiprocessing as mp

global progress_bar
progress_bar = tqdm.tqdm(total=len(export_meta_dict))

def update_progress_bar(_):
    progress_bar.update()


pool = mp.Pool(processes=64)
for k, v in export_meta_dict.items():
    pool.apply_async(work, (
        k,
        v,
        relative_root,
        align_images_dir,
        align_parsing_dir,
        head_images_dir,
        head_parsing_dir,
    ), callback=update_progress_bar)
pool.close()
pool.join()
progress_bar.close()

100%|██████████| 148320/148320 [50:33<00:00, 48.90it/s]  


In [17]:
print('export_start_idx', export_start_idx)

export_start_idx 273571
